## Generating load scenarios using PGscen ##

In this notebook we will use PGscen to create demand scenarios for the load zones in the Texas 7k power grid. This can also be done using the command line tool ```pgscen-load``` installed as part of PGscen.

In [ ]:
start_date = '2018-09-15'
!pgscen-load {start_date} 3 -vv

Here we will go through and explain the steps carried out by the above tool in generating scenarios for a single day, as well as show some of the characteristics of the model that PGscen implements.

We begin by finding the folder in which input datasets are located, and where the output scenarios will be saved. We then choose the date for which scenarios will be generated. Although scenarios are generated starting at midnight local time, we normalize our data to UTC time, and our scenarios start at 6am UTC.

In [ ]:
from pathlib import Path

cur_path = Path("day-ahead_load.ipynb").parent.resolve()
data_dir = Path(cur_path, '..', "data").resolve()
print("The data folder we will use is: {}".format(data_dir))

import pandas as pd

scenario_count = 1000
scen_start_time = pd.to_datetime(' '.join([start_date, "06:00:00"]), utc=True)
print("Scenarios will start at: {}".format(scen_start_time))

The next step is to load the actual demand observed for each load zone and the day-ahead forecasted demand. These two datasets are then split according to whether they came from before the day we want to generate scenarios for (```hist```) or after the scenario day (```futures```).

In [ ]:
from pgscen.command_line import (
    load_load_data, split_actuals_hist_future, split_forecasts_hist_future)
from IPython.display import display


scen_timesteps = pd.date_range(start=scen_start_time, periods=24, freq='H')

load_zone_actual_df, load_zone_forecast_df = load_load_data()

(load_zone_actual_hists,
     load_zone_actual_futures) = split_actuals_hist_future(
            load_zone_actual_df, scen_timesteps)

(load_zone_forecast_hists,
     load_zone_forecast_futures) = split_forecasts_hist_future(
            load_zone_forecast_df, scen_timesteps)

print("LOAD ACTUALS")
display(load_zone_actual_df.iloc[:5, :].round(1))
print("")
print("LOAD FORECASTS")
display(load_zone_forecast_df.iloc[:5, :].round(1))
print("")

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams['figure.figsize'] = [19, 11]

fig, (hist_ax, future_ax) = plt.subplots(figsize=(17, 6), nrows=1, ncols=2)
title_args = dict(weight='semibold', size=27)
actual_clr, fcst_clr = "#430093", "#D9C800"
plt_asset = 'North_Central'

hist_ax.set_title("History", **title_args)
hist_ax.plot(load_zone_actual_hists[plt_asset][-250:], c=actual_clr)
hist_ax.plot(load_zone_forecast_hists['Forecast_time'][-250:],
             load_zone_forecast_hists[plt_asset][-250:],
             c=fcst_clr)
hist_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

future_ax.set_title("Future", **title_args)
future_ax.plot(load_zone_actual_futures[plt_asset][:250], c=actual_clr)
future_ax.plot(load_zone_forecast_futures['Forecast_time'][:250],
               load_zone_forecast_futures[plt_asset][:250], c=fcst_clr)
future_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

lgnd_ptchs = [Patch(color=actual_clr, alpha=0.53, label="Actuals"),
              Patch(color=fcst_clr, alpha=0.53, label="Forecasts")]

_ = fig.legend(handles=lgnd_ptchs, frameon=False, fontsize=17, ncol=2, loc=8,
               bbox_to_anchor=(0.5, -0.04), handletextpad=0.7)

We use the historical data to train our scenario GEMINI model, which is handled by our scenario engine. This model will fit a covariance matrix for the set of assets (in this case, load zones), and another covariance matrix for the set of time points for the day.

In [ ]:
from pgscen.engine import GeminiEngine

ge = GeminiEngine(load_zone_actual_hists, load_zone_forecast_hists,
                  scen_start_time, asset_type='load')
ge.fit(asset_rho=0.05, horizon_rho=0.05)

display(ge.model.asset_cov)
display(ge.model.horizon_cov.iloc[:7, :7])

We see, for example, that using a stronger regularization penalty for our LASSO model when fitting the model results in sparser covariance matrices.

In [ ]:
import seaborn as sns
from ipywidgets import interact, FloatText
from pgscen.utils.plot_utils import get_clustermat, cov_cmap


def plot_zone_covars(rho):
    ge.fit(asset_rho=rho, horizon_rho=rho)

    fig, axarr = plt.subplots(figsize=(15, 7), nrows=1, ncols=2)
    asset_clust = get_clustermat(ge.model.asset_cov)
    asset_var = asset_clust.iloc[0, 0]
    time_clust = get_clustermat(ge.model.horizon_cov)
    time_var = time_clust.iloc[0, 0]

    sns.heatmap(asset_clust, ax=axarr[0],
                cmap=cov_cmap, vmin=-asset_var, vmax=asset_var, square=True)
    sns.heatmap(time_clust, ax=axarr[1],
                cmap=cov_cmap, vmin=-time_var, vmax=time_var, square=True)

    asset_lbls = [asset.replace('_', ' ') for asset in asset_clust.index]
    axarr[0].set_xticklabels(asset_lbls, rotation=31, ha='right', size=17)
    axarr[0].set_yticklabels(asset_lbls, size=17)

    time_lbls = [ge.scen_timesteps[int(lag.split('_')[1])].strftime("%H:%M")
                 for lag in time_clust.index]
    axarr[1].set_xticklabels(time_lbls, rotation=90, size=13)
    axarr[1].set_yticklabels(time_lbls, size=13)

    axarr[0].set_title("Load Covariance\n", **title_args)
    axarr[1].set_title("Timestep Covariance\n", **title_args)
    plt.tight_layout(w_pad=7)    

w = FloatText(value=0.05,
              layout={'align_self': 'center'}, style={'description_width': 'initial'},
              disabled=False)
_ = interact(plot_zone_covars, rho=w)

We can now use this fitted model to produce scenarios. This is done by producing deviations from the forecasted data for the given day using the distributions whose parameters were determined during the fitting step.

In [ ]:
ge.create_scenario(scenario_count, load_zone_forecast_futures,
                   bin_width_ratio=0.1, min_sample_size=400)

display(ge.scenarios['load']['Coast'].round(1))

In [ ]:
from ipywidgets import Dropdown


def plot_zone_scenarios(plt_zone):
    fig, ax = plt.subplots(figsize=(15, 7))

    for i in range(scenario_count):
        plt.plot(ge.scenarios['load'].iloc[i][plt_zone],
                 c='black', alpha=0.13, lw=0.2)

    plt_fcst = ge.forecasts['load'][plt_zone]
    plt.plot(plt_fcst, c=fcst_clr, alpha=0.47, lw=4.1)
    plt.plot(load_zone_actual_futures.loc[plt_fcst.index, plt_zone],
             c=actual_clr, alpha=0.47, lw=4.1)

    quant_df = ge.scenarios['load'][plt_zone].quantile([0.25, 0.75])
    ax.fill_between(quant_df.columns, quant_df.iloc[0], quant_df.iloc[1],
                    color='red', alpha=0.31)

    lgnd_ptchs = [Patch(color='black', alpha=0.23, label="Scenarios"),
                  Patch(color='red', alpha=0.41, label="Interquartile Range"),
                  Patch(color=fcst_clr, alpha=0.81, label="Forecast"),
                  Patch(color=actual_clr, alpha=0.81, label="Actual")]

    _ = plt.legend(handles=lgnd_ptchs, frameon=False, fontsize=17, ncol=4, loc=8,
                   bbox_to_anchor=(0.5, -0.17), handletextpad=0.7)

w = Dropdown(options=ge.asset_list, description="Scenarios for zone:",
             layout={'align_self': 'center'}, style={'description_width': 'initial'},
             disabled=False)
_ = interact(plot_zone_scenarios, plt_zone=w)

Our final step is to save the generated scenarios to file. We include the actual and forecasted load demands in the saved data to facilitate downstream analyses.

In [ ]:
ge.write_to_csv(data_dir, load_zone_actual_futures, write_forecasts=True)
!ls 20180815/load